In [1]:
import argparse
import os
os.umask(0)
# os.environ["MKL_NUM_THREADS"] = "1"
# os.environ["NUMEXPR_NUM_THREADS"] = "1"
# os.environ["OMP_NUM_THREADS"] = "1"
import pickle
import sys
from importlib import import_module

import torch
from torch.utils.data import DataLoader, Sampler
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm

from data import ArgoTestDataset
from torch.utils.data import Sampler, DataLoader
from utils import Logger, load_pretrain




In [2]:


# Import all settings for experiment.

model = import_module("lanegcn")
config, _, collate_fn, net, loss, post_process, opt = model.get_model()

# load pretrain model
# ckpt_path = "/mnt/lustre/tangxiaqiang/Code/LaneGCN/36.000.ckpt"
# if not os.path.isabs(ckpt_path):
#     ckpt_path = os.path.join(config["save_dir"], ckpt_path)
# ckpt = torch.load(ckpt_path, map_location=lambda storage, loc: storage)

# # load a ddp model 
# from collections import OrderedDict
# new_state_dict = OrderedDict()  
# for k, v in ckpt["state_dict"].items():
#     name = k[7:] # remove `module.`
#     new_state_dict[name] = v
# net.load_state_dict(new_state_dict,strict=True)


ckpt_path = "/mnt/lustre/tangxiaqiang/Code/LaneGCN/pretrain.ckpt"
if not os.path.isabs(ckpt_path):
    ckpt_path = os.path.join(config["save_dir"], ckpt_path)
ckpt = torch.load(ckpt_path, map_location=lambda storage, loc: storage)
net.load_state_dict(ckpt["state_dict"],strict=True)

# from collections import OrderedDict
# new_state_dict = OrderedDict()  
# ckpt_path = "/mnt/lustre/tangxiaqiang/Code/LaneGCN/model.ckpt"
# ckpt = torch.load(ckpt_path, map_location=lambda storage, loc: storage)
# for k, v in ckpt.items():
#     name = k[7:] # remove `module.`
#     new_state_dict[name] = v
# net.load_state_dict(new_state_dict,strict=True)

<All keys matched successfully>

In [5]:
config


{'display_iters': 400,
 'val_iters': 1608,
 'save_freq': 1.0,
 'epoch': 0,
 'horovod': True,
 'opt': 'adam',
 'num_epochs': 36,
 'lr': [0.004, 0.0004],
 'lr_epochs': [32],
 'lr_func': <utils.StepLR at 0x7fe007a49a10>,
 'save_dir': '/mnt/lustre/tangxiaqiang/Code/LaneGCN/results/lanegcn',
 'batch_size': 128,
 'val_batch_size': 32,
 'workers': 0,
 'val_workers': 0,
 'train_split': '/mnt/lustre/tangxiaqiang/Code/LaneGCN/dataset/train/data',
 'val_split': '/mnt/lustre/tangxiaqiang/Code/LaneGCN/dataset/val/data',
 'test_split': '/mnt/lustre/tangxiaqiang/Code/LaneGCN/dataset/test_obs/data',
 'preprocess': True,
 'preprocess_train': '/mnt/lustre/tangxiaqiang/Code/LaneGCN/dataset/preprocess/train_crs_dist6_angle90.p',
 'preprocess_val': '/mnt/lustre/tangxiaqiang/Code/LaneGCN/dataset/preprocess/val_crs_dist6_angle90.p',
 'preprocess_test': '/mnt/lustre/tangxiaqiang/Code/LaneGCN/dataset/preprocess/test_test.p',
 'rot_aug': False,
 'pred_range': [-100.0, 100.0, -100.0, 100.0],
 'num_scales': 6,
 '

In [3]:
config['preprocess']=True
# Data loader for evaluation

Dataset=_
dataset = Dataset(config["val_split"], config, train=False)
val_loader = DataLoader(
    dataset,
    batch_size=config["val_batch_size"],
    num_workers=config["val_workers"],
    collate_fn=collate_fn,
    shuffle=True,
    pin_memory=True,
)

# batch=next(iter(val_loader))
# batch
##test dataloader
# dataset = ArgoTestDataset("test", config, train=False)
# test_loader = DataLoader(
#     dataset,
#     batch_size=128,#config["val_batch_size"],
#     num_workers=config["val_workers"],
#     collate_fn=collate_fn,
#     shuffle=True,
#     pin_memory=True,
# )


In [4]:
#val
preds = {}
gts = {}
cities = {}
net.cuda()
metrics = dict()
for ii, data in tqdm(enumerate(val_loader)):
    data = dict(data)
    with torch.no_grad():
        output = net(data)
        results = [x[0:1].detach().cpu().numpy() for x in output["reg"]]

        loss_out = loss(output, data)
        print(loss_out)
        post_out = post_process(output, data)
        post_process.append(metrics, loss_out, post_out)
        post_process.display(metrics, 0, 0, 0)

    # for i, (argo_idx, pred_traj) in enumerate(zip(data["argo_id"], results)):
    #     preds[argo_idx] = pred_traj.squeeze()
    #     cities[argo_idx] = data["city"][i]
    #     gts[argo_idx] = data["gt_preds"][i][0] if "gt_preds" in data else None

1it [01:43, 103.92s/it]

{'cls_loss': tensor(211.7278, device='cuda:0'), 'num_cls': 797, 'reg_loss': tensor(61167.3672, device='cuda:0'), 'num_reg': 12306, 'loss': tensor(5.2362, device='cuda:0')}
Epoch 0.000, lr 0.00000, time 0.00
Wed Jul 27 08:46:15 2022
loss 5.2362 0.2657 4.9705, ade1 10.7087, fde1 19.6735, ade 8.4153, fde 13.9891



2it [01:44, 42.95s/it] 

{'cls_loss': tensor(134.3954, device='cuda:0'), 'num_cls': 720, 'reg_loss': tensor(70207.8438, device='cuda:0'), 'num_reg': 11848, 'loss': tensor(6.1124, device='cuda:0')}
Epoch 0.000, lr 0.00000, time 0.00
Wed Jul 27 08:46:15 2022
loss 5.6672 0.2282 5.4391, ade1 12.1658, fde1 23.2308, ade 8.3240, fde 14.0980



3it [01:44, 23.70s/it]

...

Wed Jul 27 08:52:04 2022
loss nan nan nan, ade1 nan, fde1 nan, ade nan, fde nan

{'cls_loss': tensor(nan, device='cuda:0'), 'num_cls': 0, 'reg_loss': tensor(nan, device='cuda:0'), 'num_reg': 11425, 'loss': tensor(nan, device='cuda:0')}
Epoch 0.000, lr 0.00000, time 0.00


1233it [07:33,  2.71it/s]

Wed Jul 27 08:52:04 2022
loss nan nan nan, ade1 nan, fde1 nan, ade nan, fde nan



1234it [07:33,  2.72it/s]

{'cls_loss': tensor(nan, device='cuda:0'), 'num_cls': 0, 'reg_loss': tensor(nan, device='cuda:0'), 'num_reg': 4518, 'loss': tensor(nan, device='cuda:0')}
Epoch 0.000, lr 0.00000, time 0.00
Wed Jul 27 08:52:05 2022
loss nan nan nan, ade1 nan, fde1 nan, ade nan, fde nan



In [ ]:
output = net(data)

In [ ]:

val_out=output
val_data=data
val_out.keys()
len(val_data['orig']) #dict_keys(['city', 'orig', 'gt_preds', 'has_preds', 'theta', 'rot', 'feats', 'ctrs', 'graph'])

cls,reg=val_out["cls"], val_out["reg"]
reg = torch.cat([x for x in reg], 0)
print(f"reg shape{reg.shape}")
gt_preds=torch.cat([x for x in val_data['gt_preds']], 0)
print(f"gt_preds.shape{gt_preds.shape}")

print(f"reg_e.g.{reg[0][0]}")
print(f"gt_e.g.{gt_preds[0]}")



In [ ]:
# val_data["orig"]
# orig = torch.cat([x for x in val_data["orig"]], 0)

# has_preds=val_data["has_preds"]
# has_preds=torch.cat([x for x in val_data["orig"]], 0)
# has_preds.shape

graph=val_data["graph"]
graph[0].keys()

In [ ]:
# begin inference
preds = {}
gts = {}
cities = {}
net.cuda()
metrics = dict()
for ii, data in tqdm(enumerate(data_loader)):
    data = dict(data)
    with torch.no_grad():
        output = net(data)
        results = [x[0:1].detach().cpu().numpy() for x in output["reg"]]


    for i, (argo_idx, pred_traj) in enumerate(zip(data["argo_id"], results)):
        preds[argo_idx] = pred_traj.squeeze()
        cities[argo_idx] = data["city"][i]
        gts[argo_idx] = data["gt_preds"][i][0] if "gt_preds" in data else None

# save for further visualization
res = dict(
    preds = preds,
    gts = gts,
    cities = cities,
)
# torch.save(res,f"{config['save_dir']}/results.pkl")

# evaluate or submit
if args.split == "val":
    # for val set: compute metric
    from argoverse.evaluation.eval_forecasting import (
        compute_forecasting_metrics,
    )
    # Max #guesses (K): 6
    _ = compute_forecasting_metrics(preds, gts, cities, 6, 30, 2)
    # Max #guesses (K): 1
    _ = compute_forecasting_metrics(preds, gts, cities, 1, 30, 2)
else:
    # for test set: save as h5 for submission in evaluation server
    from argoverse.evaluation.competition_util import generate_forecasting_h5
    generate_forecasting_h5(preds, f"{config['save_dir']}/submit.h5")  # this might take awhile
    print("----------------finish generate---------------")